<a href="https://colab.research.google.com/github/juhumkwon/source_code/blob/main/Faster_R_CNN(pre_trained).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:

import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import matplotlib.pyplot as plt
import cv2
import urllib.request

# 1. 테스트 이미지 다운로드 및 전처리
# (예제에서는 위키미디어에 있는 고양이 이미지를 사용합니다.)
url = "https://upload.wikimedia.org/wikipedia/commons/6/60/Felis_silvestris_catus.jpg"
resp = urllib.request.urlopen(url)
image_data = np.asarray(bytearray(resp.read()), dtype="uint8")
image = cv2.imdecode(image_data, cv2.IMREAD_COLOR)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# 모델의 입력 크기에 맞게 이미지 크기 조정 (모델은 640x640 입력을 기대)
input_size = 640
image_resized = cv2.resize(image, (input_size, input_size))
image_input = image_resized.astype(np.float32) / 255.0
image_input = np.expand_dims(image_input, axis=0)  # 배치 차원 추가

# 2. TensorFlow Hub에서 Faster R-CNN 모델 로드 (2-Stage Detector)
# 이 모델은 Region Proposal Network(RPN)와 후속 분류 및 박스 회귀 단계로 구성됩니다.
detector = hub.load("https://tfhub.dev/tensorflow/faster_rcnn/resnet50_v1_640x640/1")

# 3. 모델 추론 (객체 탐지)
result = detector(image_input)

# 결과를 numpy 배열로 변환
result = {key: value.numpy() for key, value in result.items()}

# 일반적으로 모델 출력에는 다음 항목들이 포함됩니다:
#   - "detection_boxes": [1, num_detections, 4] (ymin, xmin, ymax, xmax) - 정규화 좌표
#   - "detection_scores": [1, num_detections]
#   - "detection_class_entities": [1, num_detections] (클래스 이름, bytes 타입일 수 있음)
#   - "num_detections": 실제 검출 개수

detection_boxes = result["detection_boxes"][0]
detection_scores = result["detection_scores"][0]
detection_classes = result["detection_class_entities"][0]

# 4. 결과 시각화 (원본 이미지에 바운딩 박스와 클래스 정보 표시)
fig, ax = plt.subplots(1, figsize=(12, 8))
ax.imshow(image)

height, width, _ = image.shape
score_threshold = 0.5  # 일정 점수 미만의 검출은 무시

for i in range(detection_boxes.shape[0]):
    score = detection_scores[i]
    if score < score_threshold:
        continue
    box = detection_boxes[i]
    ymin, xmin, ymax, xmax = box
    # 정규화된 좌표를 원본 이미지 크기에 맞게 변환
    left, top = int(xmin * width), int(ymin * height)
    right, bottom = int(xmax * width), int(ymax * height)
    # 바운딩 박스 그리기
    rect = plt.Rectangle((left, top), right - left, bottom - top, fill=False, color='red', linewidth=2)
    ax.add_patch(rect)
    # 클래스 이름 (bytes 타입일 수 있으므로 decode)
    label = detection_classes[i]
    if isinstance(label, bytes):
        label = label.decode("ascii")
    ax.text(left, top, f"{label}: {score:.2f}", fontsize=12, color='white',
            bbox=dict(facecolor='red', alpha=0.5))

plt.axis('off')
plt.show()

HTTPError: HTTP Error 404: Not Found